## visitseoul 크롤링
* 여행지 카테고리의 명소/쇼핑/엔터테인먼트/자연 그룹 크롤링
* 크롤링 항목 : 장소명 / 평점 / item_code / 주소 

In [136]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time

from tqdm import tqdm, trange #대량데이터 처리시 진행상황을 표시합니다.
import random
import re

In [ ]:
driver = webdriver.Chrome()

### 장소명 / 평점 / 싸이트내 장소코드
*싸이트내 장소코드 주소 크롤링시 필요한 항목

In [4]:
#명소
title = []
title_no = []
point = []

for pnum in range(1,35): #카테고리별 마지막 페이지 범위까지 지정해 줌.
    print(pnum, end="/")
    base_url = f"https://korean.visitseoul.net/attractions?curPage={pnum}"
    driver.get(base_url)

    html = driver.page_source
    soup = bs(html, 'lxml') 
    entire=soup.find('ul', class_='article-list')
    place_list=entire.find_all('li' )
    for name in place_list: 
        title.append(name.select_one("div > span.title").text)
        title_no.append(name.a.attrs['href'].split('/')[3])
        img = (name.select_one("div > span.trip-ico > img"))
        point.append(img)

1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/

In [65]:
print(len(title))
print(len(point))
print(len(title_no))

271
271
271


In [5]:
#point 리스트로 만들어서 평점 데이터 가져오기
def get_grade(bs_tag):
    point_list = []
    for i in point:
        point_list.append(str(i))

    grade = []
    for j in point_list:
        if j == None:
            continue
        else:
            grade.append(j[10:16])
    
    return grade

In [6]:
user_point = get_grade(point)
print(len(user_point))

271


In [7]:
#크롤링한 데이터 딕셔너리에 담아주기
visit_sel = {}
for num in range(len(title)):
    #print(num, end='')
    visit_sel[title[num]] = [user_point[num],title_no[num]]
    #print(user_point[num])
    
    
visit_sel

{'양화진외국인선교사묘원': ['평점:5.0', '1153'],
 '호림박물관 (신사분관)': ['평점:5.0', '5548'],
 '갤러리 학고재': ['평점:5.0', '1872'],
 '송은아트큐브 (舊 송은갤러리)': ['평점:5.0', '3778'],
 '김종영 미술관': ['평점:5.0', '2066'],
 '경희대학교 한의학역사박물관': ['평점:5.0', '6889'],
 '경복궁': ['평점:4.5', '72'],
 '창덕궁': ['평점:4.5', '295'],
 '전쟁기념관': ['평점:4.5', '859'],
 '국립중앙박물관': ['평점:4.5', '433'],
 '봉은사 ': ['평점:4.5', '411'],
 '명동성당': ['평점:4.5', '4037'],
 '조계사': ['평점:4.5', '2587'],
 '예술의 전당 한가람미술관': ['평점:4.5', '4325'],
 '롯데월드타워': ['평점:4.5', '21278'],
 '삼성미술관 리움 ': ['평점:4.5', '1232'],
 '창경궁': ['평점:4.5', '297'],
 '몽촌역사관': ['평점:4.5', '5507'],
 '삼청동 골목길': ['평점:4.5', '2121'],
 '서대문형무소역사관': ['평점:4.5', '1833'],
 '서울역사박물관': ['평점:4.5', '2723'],
 '국립현대미술관 서울관 갤러리아트존': ['평점:4.5', '16792'],
 '국립현대미술관 서울관': ['평점:4.5', '11659'],
 '서울 한양도성': ['평점:4.5', '90'],
 '경희대학교 중앙박물관': ['평점:4.5', '6927'],
 '서울중앙성원': ['평점:4.5', '4867'],
 '아라리오뮤지엄 인 스페이스': ['평점:4.5', '20731'],
 '롯데월드 민속박물관': ['평점:4.5', '5815'],
 '고려대학교 본관 및 구 중앙도서관': ['평점:4.5', '2962'],
 '국립서울현충원': ['평점:4.5', '1965']

In [8]:
# 중복된 key값이 같은 데이터는 합쳐짐
print(len(visit_sel.keys()))
print(len(visit_sel.values()))

246
246


In [9]:
import pandas as pd

new_data = pd.DataFrame(visit_sel,index=['평점', 'NO'] )#데이터 프레임으로 전환,
df2 = new_data.head().T
final_data = df2.reset_index()
final_data

index      평점     NO
0         양화진외국인선교사묘원  평점:5.0   1153
1        호림박물관 (신사분관)  평점:5.0   5548
2             갤러리 학고재  평점:5.0   1872
3    송은아트큐브 (舊 송은갤러리)  평점:5.0   3778
4             김종영 미술관  평점:5.0   2066
..                ...     ...    ...
241             종로양복점          24708
242              국선옻칠          24705
243              구하산방          24703
244         동대문디자인플라자          24679
245               코엑스          24663

[246 rows x 3 columns]

In [11]:
# 엑셀로 저장하기
final_data.to_excel(r'C:\Users\sunny\Desktop\DT\8주차_팀과제_서울관광객EDA\data\visitseoul.xlsx', index = False) 

In [12]:
df_data = pd.read_excel(r'C:\Users\sunny\Desktop\DT\8주차_팀과제_서울관광객EDA\data\visitseoul.xlsx')
df_data

index      평점     NO
0         양화진외국인선교사묘원  평점:5.0   1153
1        호림박물관 (신사분관)  평점:5.0   5548
2             갤러리 학고재  평점:5.0   1872
3    송은아트큐브 (舊 송은갤러리)  평점:5.0   3778
4             김종영 미술관  평점:5.0   2066
..                ...     ...    ...
241             종로양복점     NaN  24708
242              국선옻칠     NaN  24705
243              구하산방     NaN  24703
244         동대문디자인플라자     NaN  24679
245               코엑스     NaN  24663

[246 rows x 3 columns]

In [13]:
#평점 데이터 숫자값만 남기기
a = df_data['평점'].str.split(':')
new_col = []
for i in a:
    i = str(i)
    #print(i)
    new_col.append(i[8:11])  

In [17]:
df_data['평점'] = new_col
df_data

index   평점     NO
0         양화진외국인선교사묘원  5.0   1153
1        호림박물관 (신사분관)  5.0   5548
2             갤러리 학고재  5.0   1872
3    송은아트큐브 (舊 송은갤러리)  5.0   3778
4             김종영 미술관  5.0   2066
..                ...  ...    ...
241             종로양복점       24708
242              국선옻칠       24705
243              구하산방       24703
244         동대문디자인플라자       24679
245               코엑스       24663

[246 rows x 3 columns]

In [18]:
df_data.to_excel(r'C:\Users\sunny\Desktop\DT\8주차_팀과제_서울관광객EDA\data\visitseoul.xlsx', index = False)

In [20]:
df_data = pd.read_excel(r'C:\Users\sunny\Desktop\DT\8주차_팀과제_서울관광객EDA\data\visitseoul.xlsx')
df_data

index   평점     NO
0         양화진외국인선교사묘원  5.0   1153
1        호림박물관 (신사분관)  5.0   5548
2             갤러리 학고재  5.0   1872
3    송은아트큐브 (舊 송은갤러리)  5.0   3778
4             김종영 미술관  5.0   2066
..                ...  ...    ...
241             종로양복점  NaN  24708
242              국선옻칠  NaN  24705
243              구하산방  NaN  24703
244         동대문디자인플라자  NaN  24679
245               코엑스  NaN  24663

[246 rows x 3 columns]

In [23]:
df_data.columns = ['방문지','평점','NO']
df_data

방문지   평점     NO
0         양화진외국인선교사묘원  5.0   1153
1        호림박물관 (신사분관)  5.0   5548
2             갤러리 학고재  5.0   1872
3    송은아트큐브 (舊 송은갤러리)  5.0   3778
4             김종영 미술관  5.0   2066
..                ...  ...    ...
241             종로양복점  NaN  24708
242              국선옻칠  NaN  24705
243              구하산방  NaN  24703
244         동대문디자인플라자  NaN  24679
245               코엑스  NaN  24663

[246 rows x 3 columns]

## 주소 크롤링

In [24]:
baseUrl = 'https://korean.visitseoul.net/attractions/'
add = []
for n in range(70): # 4.0 이상의 데이터만 확인하여 수집함.len(df_data['방문지'])
    print(n, end='\n')
    url = f'{baseUrl}{df_data.방문지[n]}_/{df_data.NO[n]}'
    driver_vs = webdriver.Chrome()
    driver_vs.get(url)
    html = driver_vs.page_source
    soup = bs(html, 'lxml') 
    try:
        add.append(soup.select_one('div.detail-map-wrap > div.detail-map-infor > dl:nth-child(1) > dd').text)
    #unicodedata.normalize('NFC', content)
        print(add[n])
    except:
        add = ''

0

					04084  서울 마포구 양화진길 46 (합정동, 양화진홍보관)  
1

					06021  서울 강남구 도산대로 317 (신사동, 호림아트센터 1빌딩)  
2

					03053  서울특별시 종로구 삼청로 50 (소격동)  
3

					06182  서울 강남구 영동대로 421 (대치동, 삼탄빌딩)  
4

					03004  서울특별시 종로구 평창32길 30  (평창동)  
5

					02453  서울 동대문구 경희대로 26-6 (회기동, 경희대학교)  
6

					03045  서울 종로구 사직로 161 (세종로, 경복궁)  
7

					03072  서울 종로구 율곡로 99 (와룡동, 창덕궁)  
8

					04353  서울 용산구 이태원로 29 (용산동1가, 전쟁기념관)  
9

					04383  서울 용산구 서빙고로 137 (국립중앙박물관)  
10

					06087  서울 강남구 봉은사로 531 (봉은사)  
11

					04537  서울 중구 명동길 74 (명동성당)  
12

					03144  서울특별시 종로구 우정국로 55 (조계사)  
13

					06757  서울특별시 서초구 남부순환로 2406  (서초동)  
14

					05551  서울 송파구 올림픽로 300 (신천동)  
15

					04348  서울 용산구 이태원로55길 60-16 (한남동)  
16

					03072  서울 종로구 창경궁로 185 (와룡동, 창경궁)  
17

					05540  서울 송파구 올림픽로 424 (몽촌역사관)  
18

					  서울특별시 종로구 가회동 일대  
19

					03732  서울 서대문구 통일로 251 (현저동, 독립공원)  
20

					03177  서울 종로구 새문안로 55 (신문로2가, 서울역사박물관)  
21

					03062  서울 종로구 삼청로 30 (소격동)  국립현대미술관 
22

					03062  서울 종로구 삼청로 30 (소격동)  
23



Exception ignored in: <function Service.__del__ at 0x000001EBB9424790>
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "c:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "c:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    if not self.is_connectable():
  File "c:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 115, in is_connectable
    return utils.is_connectable(self.port)
  File "c:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\utils.py", line 106, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "c:\ProgramData\Anaconda3\lib\socket.py", line 796, in create_connection
    sock.connect(sa)
KeyboardInterrupt: 


					03131  서울 종로구 삼일대로 464 (운현궁)  
71


WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=91.0.4472.124)


In [105]:
print(add[0:10])
print(type(add))

['\n\t\t\t\t\t04084\xa0\xa0서울 마포구 양화진길 46 (합정동, 양화진홍보관)\xa0\xa0', '\n\t\t\t\t\t06021\xa0\xa0서울 강남구 도산대로 317 (신사동, 호림아트센터 1빌딩)\xa0\xa0', '\n\t\t\t\t\t03053\xa0\xa0서울특별시 종로구 삼청로 50 (소격동)\xa0\xa0', '\n\t\t\t\t\t06182\xa0\xa0서울 강남구 영동대로 421 (대치동, 삼탄빌딩)\xa0\xa0', '\n\t\t\t\t\t03004\xa0\xa0서울특별시 종로구 평창32길 30  (평창동)\xa0\xa0', '\n\t\t\t\t\t02453\xa0\xa0서울 동대문구 경희대로 26-6 (회기동, 경희대학교)\xa0\xa0', '\n\t\t\t\t\t03045\xa0\xa0서울 종로구 사직로 161 (세종로, 경복궁)\xa0\xa0', '\n\t\t\t\t\t03072\xa0\xa0서울 종로구 율곡로 99 (와룡동, 창덕궁)\xa0\xa0', '\n\t\t\t\t\t04353\xa0\xa0서울 용산구 이태원로 29 (용산동1가, 전쟁기념관)\xa0\xa0', '\n\t\t\t\t\t04383\xa0\xa0서울 용산구 서빙고로 137 (국립중앙박물관)\xa0\xa0']
<class 'list'>


In [106]:
# 정규표현식을 사용하여 ***구 정보만 추출

add_gu= []#
for gu in add:
    results = re.findall(r"\s+[가-힣]+구+\s", gu)
    ap_gu = list(results)
#     print(ap_gu[0])
    add_gu.append(ap_gu[0])

print(add_gu)

[' 마포구 ', ' 강남구 ', ' 종로구 ', ' 강남구 ', ' 종로구 ', ' 동대문구 ', ' 종로구 ', ' 종로구 ', ' 용산구 ', ' 용산구 ', ' 강남구 ', ' 중구 ', ' 종로구 ', ' 서초구 ', ' 송파구 ', ' 용산구 ', ' 종로구 ', ' 송파구 ', ' 종로구 ', ' 서대문구 ', ' 종로구 ', ' 종로구 ', ' 종로구 ', ' 종로구 ', ' 동대문구 ', ' 용산구 ', ' 종로구 ', ' 송파구 ', ' 성북구 ', ' 동작구 ', ' 성북구 ', ' 서대문구 ', ' 성북구 ', ' 용산구 ', ' 중구 ', ' 중구 ', ' 중구 ', ' 성동구 ', ' 은평구 ', ' 강북구 ', ' 서대문구 ', ' 종로구 ', ' 서초구 ', ' 서초구 ', ' 종로구 ', ' 종로구 ', ' 서초구 ', ' 종로구 ', ' 성북구 ', ' 종로구 ', ' 종로구 ', ' 중구 ', ' 종로구 ', ' 종로구 ', ' 동대문구 ', ' 강서구 ', ' 종로구 ', ' 용산구 ', ' 서초구 ', ' 중구 ', ' 종로구 ', ' 종로구 ', ' 종로구 ', ' 종로구 ', ' 중구 ', ' 영등포구 ', ' 중구 ', ' 종로구 ', ' 종로구 ', ' 강남구 ', ' 종로구 ']


In [111]:
#공백제거
add_gu2 = []
for space in add_gu:
    word = space.replace(' ', '')

    add_gu2.append(word)

print(add_gu2)

['마포구', '강남구', '종로구', '강남구', '종로구', '동대문구', '종로구', '종로구', '용산구', '용산구', '강남구', '중구', '종로구', '서초구', '송파구', '용산구', '종로구', '송파구', '종로구', '서대문구', '종로구', '종로구', '종로구', '종로구', '동대문구', '용산구', '종로구', '송파구', '성북구', '동작구', '성북구', '서대문구', '성북구', '용산구', '중구', '중구', '중구', '성동구', '은평구', '강북구', '서대문구', '종로구', '서초구', '서초구', '종로구', '종로구', '서초구', '종로구', '성북구', '종로구', '종로구', '중구', '종로구', '종로구', '동대문구', '강서구', '종로구', '용산구', '서초구', '중구', '종로구', '종로구', '종로구', '종로구', '중구', '영등포구', '중구', '종로구', '종로구', '강남구', '종로구']


In [126]:
add_df = pd.DataFrame(add_gu2)
add_df.tail(10)

0
61   종로구
62   종로구
63   종로구
64    중구
65  영등포구
66    중구
67   종로구
68   종로구
69   강남구
70   종로구

In [127]:
add_df.to_excel(r'C:\Users\sunny\Desktop\DT\8주차_팀과제_서울관광객EDA\data/visitseoul_add.xlsx', index = False) #엑셀로 저장

In [128]:
df = pd.read_excel(r'C:\Users\sunny\Desktop\DT\8주차_팀과제_서울관광객EDA\data/visitseoul_add.xlsx')
df

0
0   마포구
1   강남구
2   종로구
3   강남구
4   종로구
..  ...
66   중구
67  종로구
68  종로구
69  강남구
70  종로구

[71 rows x 1 columns]